1. Setup and Dependencies
First, let's install all the necessary libraries:

In [1]:
# Install required packages
%pip install langchain langchain-community langchain-groq faiss-cpu sentence-transformers pdf2image pytesseract beautifulsoup4 unstructured pandas python-dotenv chromadb redis Pillow

# For PDF processing
%pip install "unstructured[pdf]"
%pip install pymupdf

Note: you may need to restart the kernel to use updated packages.Collecting langchain
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Using cached click-8.1.8-py3-none-any.whl (98 kB)



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t


Using legacy 'setup.py install' for antlr4-python3-runtime, since package 'wheel' is not installed.
    Running setup.py install for antlr4-python3-runtime: started
    Running setup.py install for antlr4-python3-runtime: finished with status 'done'


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\zeesh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\zeesh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Now let's import all the required libraries:


In [4]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from typing import List, Dict, Any, Optional

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS, Chroma
from langchain_groq import ChatGroq

# Document loaders
from langchain_community.document_loaders import (
    PyPDFLoader, 
    CSVLoader, 
    WebBaseLoader, 
    DirectoryLoader, 
    TextLoader, 
    UnstructuredMarkdownLoader,
    DataFrameLoader,
    AsyncChromiumLoader
)

# For web scraping enhancement
from langchain_community.document_transformers import BeautifulSoupTransformer

# Load environment variables
load_dotenv()

False

2. Create Helper Functions for Document Loading
Let's create helper functions for loading different types of documents:

In [ ]:
def load_pdf(file_path: str) -> List[Dict[str, Any]]:
    """
    Load a PDF file and return its contents as documents.
    """
    loader = PyPDFLoader(file_path)
    return loader.load()

def load_text(file_path: str) -> List[Dict[str, Any]]:
    """
    Load a text file and return its contents as a document.
    """
    loader = TextLoader(file_path)
    return loader.load()

def load_csv(file_path: str) -> List[Dict[str, Any]]:
    """
    Load a CSV file and return its contents as documents.
    """
    loader = CSVLoader(file_path)
    return loader.load()

def load_website(url: str) -> List[Dict[str, Any]]:
    """
    Load content from a website URL and return as documents.
    """
    loader = WebBaseLoader(url)
    return loader.load()

def load_javascript_website(url: str) -> List[Dict[str, Any]]:
    """
    Load content from a JavaScript-heavy website using a headless browser.
    Requires Chrome/Chromium to be installed.
    """
    loader = AsyncChromiumLoader([url])
    html_documents = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    return bs_transformer.transform_documents(html_documents)

def load_directory(directory_path: str, glob_pattern: str = "**/*") -> List[Dict[str, Any]]:
    """
    Load all files matching the glob pattern from a directory.
    """
    # Try to determine the appropriate loader based on file extension
    loaders = {
        ".pdf": PyPDFLoader,
        ".txt": TextLoader,
        ".csv": CSVLoader,
        ".md": UnstructuredMarkdownLoader
    }
    
    documents = []
    for ext, loader_cls in loaders.items():
        try:
            dir_loader = DirectoryLoader(
                directory_path, 
                glob=f"**/*{ext}", 
                loader_cls=loader_cls
            )
            documents.extend(dir_loader.load())
            print(f"Loaded {ext} files")
        except Exception as e:
            print(f"Error loading {ext} files: {e}")
    
    return documents

def load_dataframe(df: pd.DataFrame, page_content_column: str) -> List[Dict[str, Any]]:
    """
    Load a pandas DataFrame and convert it to documents.
    """
    loader = DataFrameLoader(df, page_content_column=page_content_column)
    return loader.load()

3. Implement Text Splitting

In [ ]:
def split_documents(documents, chunk_size=1000, chunk_overlap=100):
    """
    Split documents into smaller chunks for better processing.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True,
    )
    return text_splitter.split_documents(documents)

4. Set Up Embeddings Model

In [ ]:
def get_embeddings_model():
    """
    Initialize and return the embeddings model.
    We'll use Nomic's free embedding model that can run locally.
    """
    model_name = "nomic-ai/nomic-embed-text-v1"
    
    # For very limited hardware, you could use a smaller model like:
    # model_name = "sentence-transformers/all-MiniLM-L6-v2"
    
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'cpu'},  # Use 'cuda' if you have GPU
        encode_kwargs={'normalize_embeddings': True}
    )
    return embeddings

5. Create Vector Store

In [ ]:
def create_vector_store(documents, embeddings, vector_store_type="faiss", persist_directory=None):
    """
    Create a vector store from documents using the specified embeddings.
    
    Args:
        documents: List of documents to index
        embeddings: Embedding model to use
        vector_store_type: Either "faiss" or "chroma"
        persist_directory: Directory to save the vector store (for Chroma)
    """
    if vector_store_type.lower() == "faiss":
        vector_store = FAISS.from_documents(documents, embeddings)
        return vector_store
    
    elif vector_store_type.lower() == "chroma":
        if persist_directory:
            vector_store = Chroma.from_documents(
                documents=documents,
                embedding=embeddings,
                persist_directory=persist_directory
            )
            vector_store.persist()  # Save to disk
            return vector_store
        else:
            return Chroma.from_documents(documents, embeddings)
    
    else:
        raise ValueError("Vector store type must be either 'faiss' or 'chroma'")